In [3]:
from langchain_core.tools import tool
from langchain.pydantic_v1 import BaseModel, Field

class test():
    @tool('test')
    def multiply(self, first_int: int, second_int: int) -> int:
        """Multiply two integers together.
        
        @param first_int: The first integer to multiply.
        @param second_int: The second integer to multiply.
        
        """
        return first_int * second_int




In [10]:
t = test()
tool = t.multiply

In [12]:
q = {
    "first_int": 2,
    "second_int": 3
}

tool(q)

ValidationError: 1 validation error for testSchema
self
  field required (type=value_error.missing)

In [ ]:
class test():
    def multiply(self, first_int: int, second_int: int) -> int:
        """Multiply two integers together.
        
        @param first_int: The first integer to multiply.
        @param second_int: The second integer to multiply.
        
        """
        self.hello()
        return first_int * second_int
    
    def hello(self):
        print('hello')



In [21]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

print(multiply.args_schema.schema())

test
test(first_int: int, second_int: int) -> int - Multiply two integers together.
    
    @param first_int: The first integer to multiply.
    @param second_int: The second integer to multiply.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}
{'title': 'testSchema', 'type': 'object', 'properties': {'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}, 'required': ['first_int', 'second_int']}


In [14]:
class ChemicalInput(BaseModel):
    query: str = Field(description="natural product compound string")
    
multiply.args_schema = ChemicalInput

In [15]:
print(multiply.args_schema.schema())

{'title': 'ChemicalInput', 'type': 'object', 'properties': {'query': {'title': 'Query', 'description': 'natural product compound string', 'type': 'string'}}, 'required': ['query']}


In [24]:

import logging.config
from pathlib import Path
from typing import Optional

from SPARQLWrapper import JSON, SPARQLWrapper


# The `TaxonResolver` class constructs and executes SPARQL queries to retrieve Wikidata IRIs
# based on taxon names.
class TaxonResolver():
    name = "TAXON_RESOLVER"
    description = (
        "The function takes a taxon string as input and returns the wikidata ID."
    )
    args_schema = None
    ENDPOINT_URL = "https://query.wikidata.org/sparql"
    PREFIXES = """
        PREFIX prov: <http://www.w3.org/ns/prov#>
        PREFIX pr: <http://www.wikidata.org/prop/reference/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
    """

    def __init__(self):
        super().__init__()

    @tool("TAXON_RESOLVER")
    def run(self, taxon_name: str) -> Optional[str]:
        """
        Takes a taxon name as input, builds a query, executes it, and returns
        the Wikidata IRI if found.

        Args:
          taxon_name (str): A string that represents the name of a taxon.

        Returns:
            str: A string that contains the Wikidata IRI if found, otherwise `None`.
        """
        query = self.build_query(taxon_name)
        results = self.execute_query(query)

        if results:
            try:
                bindings = results["results"]["bindings"]
                if bindings:  # Check if the list is not empty
                    return "wikidata IRI is " + bindings[0]["wikidata"]["value"]
                else:
                    
                    return None
            except KeyError:
                
                return None
        else:
            return None

    def build_query(self, taxon_name: str) -> str:
        """
        Constructs a SPARQL query to retrieve information about a taxon based on
        its name.

        Args:
          taxon_name (str): a string representing the name of a taxon.

        Returns:
          str : Sparql query that target the P225 taxon wikidata id.
        """
        return f"""
            {self.PREFIXES}
            SELECT *
            WHERE
            {{
              ?wikidata wdt:P225 "{taxon_name}" .
            }}
        """

    def execute_query(self, query: str) -> Optional[dict]:
        """
        Sends a SPARQL query to a specified endpoint URL and returns the
        results in JSON format, handling exceptions by logging errors.

        Args:
            query: Sparql query string.

        Returns:
            dict: Results in JSON format if the query is successful and results are found.
            None: If no results are found, it logs an info message and returns `None`. If there
            is an unexpected result format or an error during the process, it logs an error.
        """
        sparql = SPARQLWrapper(self.ENDPOINT_URL)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        sparql.setTimeout(600)
        
        try:
            results = sparql.queryAndConvert()
            # check if results are empty, result dict is empty or bindings are empty
            if not results or (
                results.get("results") and not results.get("results").get("bindings")
            ):
                
                return None
            
            return results
        except Exception as e:
            
            return None


In [26]:
TaxonResolver().run.description

'TAXON_RESOLVER(self, taxon_name: str) -> Optional[str] - Takes a taxon name as input, builds a query, executes it, and returns\n        the Wikidata IRI if found.\n\n        Args:\n          taxon_name (str): A string that represents the name of a taxon.\n\n        Returns:\n            str: A string that contains the Wikidata IRI if found, otherwise `None`.'